In [1]:
import pandas as pd 
import numpy as np 
import requests

### Load Streaming and Library Data 

In [2]:
#Read in our streaming files 
df_stream0 = pd.read_json('Data/StreamingHistory0.json')
df_stream1 = pd.read_json('Data/StreamingHistory1.json')

#merge the streaming dataframes 

df_streams = pd.concat([df_stream0, df_stream1])

df_streams.dtypes

endTime       object
artistName    object
trackName     object
msPlayed       int64
dtype: object

In [3]:
#Create an "UniqueID" by concatenating artistName and trackName

df_streams['UniqueID'] = df_streams['artistName'] + ": " + df_streams['trackName']

df_streams.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-10-15 00:12,THEY.,What I Know Now (feat. Wiz Khalifa),3715,THEY.: What I Know Now (feat. Wiz Khalifa)
1,2020-10-15 00:12,Sam Smith,My Oasis (feat. Burna Boy),24725,Sam Smith: My Oasis (feat. Burna Boy)
2,2020-10-15 00:12,Sam Smith,My Oasis (feat. Burna Boy),1207,Sam Smith: My Oasis (feat. Burna Boy)
3,2020-10-15 00:15,Cheat Codes,Balenciaga,173706,Cheat Codes: Balenciaga
4,2020-10-15 00:16,ARTY,Rebound - Radio Edit,85740,ARTY: Rebound - Radio Edit


In [4]:
# Read in library data 

df_library = pd.read_json('Data/YourLibrary1.json')

# Created unique column, same as for our stream dataframe

df_library['UniqueID'] = df_library['artist'] + ": " + df_library['track']

split = df_library['uri'].str.split(":", expand=True)

df_library['track_uri'] = split[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Slim,Love's Crazy,So Fly (feat. Yung Joc),spotify:track:2wmBKUx62Px6my6U1mYw18,Slim: So Fly (feat. Yung Joc),2wmBKUx62Px6my6U1mYw18
1,JACKBOYS,JACKBOYS,JACKBOYS,spotify:track:62zKJrpbLxz6InR3tGyr7o,JACKBOYS: JACKBOYS,62zKJrpbLxz6InR3tGyr7o
2,Calvin Harris,18 Months,Thinking About You (feat. Ayah Marar),spotify:track:1KtD0xaLAikgIt5tPbteZQ,Calvin Harris: Thinking About You (feat. Ayah ...,1KtD0xaLAikgIt5tPbteZQ
3,Foo Fighters,Best Of You,Best of You,spotify:track:4nUM7pGcTUK2pY1d2LybrT,Foo Fighters: Best of You,4nUM7pGcTUK2pY1d2LybrT
4,Laidback Luke,Bae (feat. Gina Turner),Bae,spotify:track:4zxhd49IbXASiDuIXmWm5o,Laidback Luke: Bae,4zxhd49IbXASiDuIXmWm5o


In [5]:
#Copy the df_streams to another df - called df_tableau

df_tableau = df_streams.copy()

#Since we created new "UniqueID" for both dataframes, it will be used for our merge

df_tableau =  pd.merge(df_tableau, df_library[['album', 'UniqueID', 'track_uri']], how='left', on=['UniqueID'])

#Drop all songs that arent in our library dataframe 

df_tableau = df_tableau[df_tableau['track_uri'].notna()]
df_tableau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9239 entries, 0 to 17981
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     9239 non-null   object
 1   artistName  9239 non-null   object
 2   trackName   9239 non-null   object
 3   msPlayed    9239 non-null   int64 
 4   UniqueID    9239 non-null   object
 5   album       9239 non-null   object
 6   track_uri   9239 non-null   object
dtypes: int64(1), object(6)
memory usage: 577.4+ KB


### Spotify Credentials

In [ ]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = ''
CLIENT_SECRET = ''

### Generate access token with requests 

In [7]:
#authentication URL
AUTH_url = 'https://accounts.spotify.com/api/token'

#POST
auth_response = requests.post(AUTH_url, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret':CLIENT_SECRET
})

#Convert the response to json 
auth_response_data = auth_response.json()

access_token = auth_response_data['access_token']


In [8]:
#Authenticating API calls

headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL for all Spotify endpoints 

BASE_url = 'https://api.spotify.com/v1/'

In [11]:
#Create blank dictionary 

track_features = {}

#Convert track_uri column to a iterable list 

track_uris = df_library['track_uri'].to_list()

#Loop through track URIs and pull audio features via the API

for t_uri in track_uris:
    track_features[t_uri] = {'popularity': 0,
                            'danceability': 0,
                            'energy': 0,
                            'speechiness': 0,
                            'instrumentalness': 0,
                            'tempo': 0}
    
    r = requests.get(BASE_url + 'tracks/' + t_uri, headers=headers)
    r = r.json()

    track_features[t_uri]['popularity'] = r['popularity']

    s = requests.get(BASE_url + 'audio-features/' + t_uri , headers=headers)
    s = s.json()

    track_features[t_uri]['dancebility'] = s['danceability']
    track_features[t_uri]['energy'] = s['energy']
    track_features[t_uri]['speechiness'] =s['speechiness']
    track_features[t_uri]['instrumentalness'] =s['instrumentalness']
    track_features[t_uri]['tempo'] = s['tempo']



In [14]:
#Convert our track_features dictionary into a dataframe 

df_features = pd.DataFrame.from_dict(track_features, orient='index')

df_features.insert(0, 'track_uri', df_features.index)
df_features.reset_index(inplace=True, drop=True)
df_features.head()

,track_uri,popularity,danceability,energy,speechiness,instrumentalness,tempo,dancebility
0,2wmBKUx62Px6my6U1mYw18,42,0,0.559,0.0846,0.000000,90.994,0.784
1,62zKJrpbLxz6InR3tGyr7o,57,0,0.130,0.0336,0.820000,123.342,0.413
2,1KtD0xaLAikgIt5tPbteZQ,66,0,0.874,0.0396,0.000412,127.985,0.725
3,4nUM7pGcTUK2pY1d2LybrT,0,0,0.944,0.0767,0.000003,130.315,0.370
4,4zxhd49IbXASiDuIXmWm5o,12,0,0.913,0.0431,0.071500,128.022,0.740


### Save our dataframes as xlsx files to upload 

In [16]:
df_tableau.to_excel('Data/myspotifylibrarystreams.xlsx')
df_features.to_excel('Data/audiofeatures.xlsx')